In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import json
!pip install jsonlines
import jsonlines
import csv

with jsonlines.open('/content/drive/My Drive/covid_project/graph_files/sampled_tweets.jsonl') as json_file, open('/content/drive/My Drive/covid_project/graph_files/tweets.csv', 'w') as txt:
  field_names = ['text']
  writer = csv.DictWriter(txt, fieldnames=field_names)

  #creating file csv with column with text of each tweet
  for line in json_file:
    
    if('retweeted_status' in line):
      x = line['retweeted_status']['full_text']
      writer.writerow({'text':x})
    else:
      x = line['full_text']
      writer.writerow({'text':x})
txt.close()

In [3]:
!pip install tweet-preprocessor
import preprocessor as p
import pandas as pd

In [4]:
tweets_csv = pd.read_csv('/content/drive/My Drive/covid_project/graph_files/tweets.csv', header = None, names=['text'], delimiter="\t", quoting=csv.QUOTE_NONE, encoding='utf-8')

tweets_csv.count()

text    44419
dtype: int64

In [5]:
#Dropping duplicates and NaN from the dataframe
tweets_csv = tweets_csv.dropna()
tweets_csv = tweets_csv.drop_duplicates()

tweets_csv.head()

,text
0,please read this thread before you judge how s...
1,A photo of Chinese medical workers and law enf...
2,"""I’m just going to assume this guy dropped a b..."
3,#Coronavirus #COVID19 is serious. We are not h...
4,"""Coronavirus Thanos"


In [6]:
tweets_csv.count()

text    30832
dtype: int64

In [7]:
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [8]:
#Tweet have to be cleaned to normal text
tweets_csv['text'] = tweets_csv.apply(preprocess_tweet, axis=1)
tweets_csv.head()

,text
0,please read this thread before you judge how s...
1,A photo of Chinese medical workers and law enf...
2,"""Im just going to assume this guy dropped a bu..."
3,"is serious. We are not hyping it. """
4,"""Coronavirus Thanos"


In [9]:
from gensim.parsing.preprocessing import remove_stopwords

In [10]:
def stopword_removal(row):
    text = row['text']
    text = remove_stopwords(text)
    return text

In [11]:
tweets_csv['text'] = tweets_csv.apply(stopword_removal, axis=1)

In [12]:
tweets_csv.head()

,text
0,read thread judge strangers navigating moment
1,A photo Chinese medical workers law enforcemen...
2,"""Im going assume guy dropped bunch F bombs mor..."
3,"serious. We hyping it. """
4,"""Coronavirus Thanos"


In [13]:
tweets_csv['text'] = tweets_csv['text'].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')

In [14]:
tweets_csv.head()

,text
0,read thread judge strangers navigating moment
1,a photo chinese medical workers law enforcemen...
2,im going assume guy dropped bunch f bombs mor...
3,serious we hyping it
4,coronavirus thanos


In [15]:
tweets_csv.to_csv('/content/drive/My Drive/covid_project/graph_files/tweets_preprocessed.csv')

In [30]:
with open('/content/drive/My Drive/covid_project/graph_files/tweets_preprocessed.csv', 'r') as inp, open('/content/drive/My Drive/covid_project/graph_files/tweets_rows_deleted.csv', 'w') as out:
    writer = csv.writer(out)
    for row in csv.reader(inp):
      if len(row[1]) > 7:
        writer.writerow(row)